### Login to Huggingface

In [1]:
from huggingface_hub import notebook_login
notebook_login()

### Load the model

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

#model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", trust_remote_code=True)

model_name = "mistralai/Mistral-7B-Instruct-v0.3"

quantization_config  = BitsAndBytesConfig(
    load_in_8bit=True, 
    llm_int8_enable_fp32_cpu_offload=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config ,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
next(model.parameters()).dtype

for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.mlp
model.layers.2.mlp.gate_proj
model.l

### Configure PEFT

In [4]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"]
)
model = get_peft_model(model, config)
model.print_trainable_parameters()


trainable params: 13,631,488 || all params: 7,261,655,040 || trainable%: 0.1877


In [5]:
len(model.peft_config)

1

### Load Dataset

In [6]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer


raw_dataset = load_dataset("json", data_files="data/formatted_data.jsonl")["train"]


tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    model_inputs = tokenizer(
        tokenizer.apply_chat_template(example["messages"], tokenize=False),
        truncation=True,
        padding="max_length",
        max_length=1024
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

tokenized_dataset = raw_dataset.map(tokenize)

tokenized_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['messages', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2
})

### Tune the model

In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,#4
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,#10
    save_steps=500,
    output_dir="./mistral-chat-lora",
    label_names=["labels"]
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args
)

trainer.train()


Step,Training Loss
1,4.483600
2,4.483600
3,4.483600


TrainOutput(global_step=3, training_loss=4.483574390411377, metrics={'train_runtime': 307.9153, 'train_samples_per_second': 0.019, 'train_steps_per_second': 0.01, 'total_flos': 262745849069568.0, 'train_loss': 4.483574390411377, 'epoch': 3.0})

### Evaluate

In [8]:
import json
from jsonschema import validate, ValidationError

# Check if a string is valid JSON
def is_valid_json(s):
    try:
        json.loads(s)
        return True
    except json.JSONDecodeError:
        return False
    

# Validate a JSON string against a schema
def is_valid_json_schema(json_str, schema):
    try:
        data = json.loads(json_str)
        validate(instance=data, schema=schema)
        return True
    except (json.JSONDecodeError, ValidationError):
        return False
    
# Compare two JSON strings at field level and match field values
def compare_json_field_values(json_ref, json_gen):
    try:
        obj1 = json.loads(json_ref)
        obj2 = json.loads(json_gen)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON input.","percentage": 0}
    matching_fields = []
    matching_values = 0
    total_fields = max(len(obj1), len(obj2))
    for key in obj1:
        if key in obj2:
            matching_fields.append(key)
            if obj1[key] == obj2[key]:
                matching_values += 1
    score = matching_values
    percentage = score / total_fields if total_fields > 0 else 0
    return {
        "matching_fields": matching_fields,
        "matching_values": score,
        "total_fields": total_fields,
        "percentage": percentage
    }



### Save the model